<a href="https://colab.research.google.com/github/Roshni-seth/Machine-Learning/blob/main/Binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle (3).json to kaggle (3).json


{'kaggle (3).json': b'{"username":"rosh05032001","key":"0684e1839ccedea0779e4cec79194e29"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [3]:
! kaggle datasets download brendan45774/test-file

Dataset URL: https://www.kaggle.com/datasets/brendan45774/test-file
License(s): CC0-1.0
  0% 0.00/11.2k [00:00<?, ?B/s]
100% 11.2k/11.2k [00:00<00:00, 31.1MB/s]


In [4]:
import numpy as np
import pandas as pd

In [6]:
df=pd.read_csv('/content/test-file.zip')[['Age','Fare','SibSp','Parch','Survived']]

In [12]:
df.dropna(inplace=True)

In [8]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,34.5,7.8292,0,0,0
1,47.0,7.0000,1,0,1
2,62.0,9.6875,0,0,0
3,27.0,8.6625,0,0,0
4,22.0,12.2875,1,1,1


In [13]:
df['family']=df['SibSp']+df['Parch']

In [14]:
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,34.5,7.8292,0,0,0,0
1,47.0,7.0000,1,0,1,1
2,62.0,9.6875,0,0,0,0
3,27.0,8.6625,0,0,0,0
4,22.0,12.2875,1,1,1,2


In [15]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [16]:
df.head()

,Age,Fare,Survived,family
0,34.5,7.8292,0,0
1,47.0,7.0000,1,1
2,62.0,9.6875,0,0
3,27.0,8.6625,0,0
4,22.0,12.2875,1,2


In [17]:
X=df.drop(columns=['Survived'])
y=df['Survived']

In [19]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
X_train.head()

,Age,Fare,family
281,0.75,13.7750,2
96,76.00,78.8500,1
341,32.00,7.5792,0
18,27.00,7.9250,1
26,22.00,61.9792,1


In [22]:
# without Binarization

clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.6268656716417911

In [23]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.5983065953654189

In [24]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [25]:
trf=ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [26]:
X_train_trf=trf.fit_transform(X_train)
X_test_trf=trf.transform(X_test)

In [28]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,0.75,13.7750
1,1.0,76.00,78.8500
2,0.0,32.00,7.5792
3,1.0,27.00,7.9250
4,1.0,22.00,61.9792
...,...,...,...
259,0.0,20.00,7.2250
260,0.0,27.00,7.8792
261,0.0,43.00,7.8958
262,0.0,16.00,7.6500


In [29]:
clf=DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2=clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6268656716417911

In [30]:
X_trf=trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.6073975044563281